In [1]:
## Deep learning
### With keras and tensorflow 
#### Got most of the code from Siddhartha's public kernel, https://www.kaggle.com/meaninglesslives/simple-neural-net-for-time-series-classification?fbclid=IwAR1UJHWeMYPjHVw4E-3HxR7gip-cSQNcyq1q1VadOATDee-Se3MpnUD8o2E

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
agg_train=pd.read_csv("train/neg_flux/agg_train.csv")
agg_train.drop("Unnamed: 0", axis=1, inplace=True)
agg_train

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,...,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det
0,615,-1100.440063,660.626343,-123.096998,-89.477524,394.109851,-0.349540,2.130510,12.845472,4.482743,...,352.0,19.0,29.0,202.114067,1.999688,35.0,4.0,2.631898,2.631898,873.7903
1,713,-14.735178,14.770886,-1.423351,-0.873033,6.471144,0.014989,0.639458,9.115748,2.359620,...,350.0,50.0,73.0,2.935177,-0.050944,199.0,8.0,14.352571,14.352571,846.8017
2,730,-19.159811,47.310059,2.267434,0.409172,8.022239,3.177854,0.695106,11.281384,2.471061,...,330.0,13.0,32.0,4.227614,-0.008131,4.0,222.0,3.580623,3.580623,78.7737
3,745,-15.494463,220.795212,8.909206,1.035895,27.558208,4.979826,0.567170,55.892746,2.555576,...,351.0,19.0,115.0,7.065548,0.008044,4.0,201.0,2.061453,2.061453,123.6872
4,1124,-16.543753,143.600189,7.145702,1.141288,20.051722,4.406298,0.695277,11.383690,2.753004,...,352.0,19.0,158.0,6.727352,0.012543,10.0,231.0,2.231855,2.231855,133.9113
5,1227,-12.695161,71.678154,0.367212,0.208109,4.962633,9.125056,0.720329,11.554160,2.596891,...,330.0,6.0,9.0,3.331646,0.012115,1.0,107.0,345.981650,345.981650,691.9633
6,1598,-13.854152,1448.715698,30.333023,0.337299,167.317942,6.278757,0.670701,11.316125,2.832957,...,352.0,17.0,331.0,13.613777,1.866834,12.0,340.0,0.799395,0.799395,15.1885
7,1632,-14.525748,34.559826,1.760026,0.623454,5.273147,2.486305,0.677202,11.346678,2.733204,...,352.0,9.0,13.0,4.187164,-0.009162,3.0,47.0,1.460600,1.460600,2.9212
8,1920,-13.367912,231.828339,19.228827,1.252908,48.004394,2.845162,0.746171,62.950779,3.000022,...,255.0,28.0,86.0,11.953869,-0.014437,4.0,114.0,1.867774,1.867774,100.8598
9,1926,-52.764721,30.658575,-0.085210,0.166696,6.288463,-1.709852,0.915097,68.224373,3.822488,...,255.0,6.0,5.0,5.460791,-0.017208,1.0,99.0,217.899950,217.899950,435.7999


In [4]:
#agg_train = pd.read_csv("train/neg_flux/agg_train.csv")
df_meta = pd.read_csv("train/neg_flux/df_meta.csv")
df_meta.drop("Unnamed: 0", axis=1, inplace=True)
dtw_train = pd.read_csv("train/neg_flux/dtw_train.csv")
max_train = pd.read_csv("train/neg_flux/max_train.csv")
min_train = pd.read_csv("train/neg_flux/min_train.csv")
mjd_diff_train = pd.read_csv("train/neg_flux/mjd_diffs.csv")
new_feats = pd.read_csv("train/neg_flux/new_feats.csv")
what = pd.read_csv("train/neg_flux/encoder_train.csv")
what = what[:7848]
deriv_train = pd.read_csv("train/neg_flux/derivatives.csv")
what.drop("Unnamed: 0", axis=1, inplace=True)
what.columns = ["Autoencoder" + str(s) for s in range(0,16)]
what["object_id"] = df_meta["object_id"]
new_feats

,object_id,new_feats0,new_feats1,new_feats2,new_feats3,new_feats4,new_feats5,new_feats6,new_feats7,new_feats8,...,new_feats62,new_feats63,new_feats64,new_feats65,new_feats66,new_feats67,new_feats68,new_feats69,new_feats70,new_feats71
0,615,659.486694,-230.643313,-310.010925,-1100.440063,0.199586,385.186195,11.395031,4.457957,3.736262,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,713,14.509829,4.163086,4.118354,-9.332252,-0.482157,3.725186,9.115748,2.275422,1.859288,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,730,11.777126,0.239381,0.106692,-12.809639,0.327705,2.822747,7.966248,2.413239,1.919590,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,745,12.491076,-0.018037,0.096550,-10.249387,0.101126,2.773487,8.552037,2.243641,1.730224,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
4,1124,11.112542,0.091815,0.528071,-16.543753,-1.378127,3.206350,10.423373,2.674462,2.157105,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
5,1227,8.092285,-0.161867,0.118753,-12.695161,-1.300771,2.907066,8.353500,2.542712,2.003150,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
6,1598,12.903332,0.170220,0.249740,-11.553727,0.237925,3.063292,10.354021,2.649326,2.121293,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
7,1632,34.559826,4.433000,2.273386,-5.074782,2.194903,6.929965,10.393328,2.662764,2.140205,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
8,1920,17.435411,-0.104133,0.171616,-13.367912,0.286568,3.978243,8.929063,3.055874,2.508360,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
9,1926,27.213717,0.478254,-0.301043,-12.523030,1.557855,5.924947,10.912610,3.834048,2.834368,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [7]:
agg_autoenc = pd.read_csv("train/neg_flux/agg_train_autoenc.csv")
agg_autoenc["object_id"] = np.array(agg_train.object_id)
agg_autoenc = agg_autoenc.drop("Unnamed: 0", axis=1)
agg_autoenc

,flux_min_encod,flux_max_encod,flux_mean_encod,flux_median_encod,flux_std_encod,flux_skew_encod,flux_diff1,flux_diff2,flux_min_0_encod,flux_min_1_encod,...,1.0__skewness,2.0__kurtosis,2.0__skewness,3.0__kurtosis,3.0__skewness,4.0__kurtosis,4.0__skewness,5.0__kurtosis,5.0__skewness,object_id
0,1.251828e+02,661.626343,434.258877,414.345398,175.302017,-0.384413,536.443535,1.235308,1.251828e+02,6.606263e+02,...,5.415376e-01,-1.400086,7.264563e-01,-1.349070,7.531514e-01,1.286499,1.780683e+00,-0.961555,9.808281e-01,615
1,4.011444e+00,13.845472,7.683726,6.344858,3.468925,0.533515,9.834028,1.279852,4.737393e+00,6.952323e+00,...,6.440304e-01,-1.302994,7.821372e-01,-1.378617,7.396648e-01,1.361939,1.805035e+00,-0.458776,1.197168e+00,713
2,2.530638e+00,15.770886,8.239407,9.134171,4.380620,0.139723,13.240248,1.606942,3.348282e+00,3.545214e+00,...,2.545466e-03,-1.958521,1.287502e-03,-1.961793,4.520380e-04,-1.954725,2.670775e-03,-1.926973,5.957179e-03,730
3,2.483539e+00,48.310059,15.505437,5.970370,15.730628,0.995825,45.826519,2.955513,3.110694e+00,3.564052e+00,...,3.325748e-02,-1.993506,1.544181e-04,-1.998700,-6.281118e-06,-2.000458,2.258646e-05,-1.999886,9.312298e-05,745
4,2.334581e+00,221.795212,87.974083,40.822909,87.853611,0.373852,219.460631,2.494606,3.093587e+00,3.585434e+00,...,8.360498e-07,-2.001766,7.164828e-07,-2.001754,-4.301664e-07,-2.001751,6.154467e-07,-2.001468,1.603322e-05,1124
5,2.631665e+00,144.600189,48.345197,11.383690,55.852943,0.732534,141.968524,2.936559,3.658313e+00,3.731154e+00,...,3.205718e-05,-2.001556,3.213107e-06,-2.001671,-8.997952e-07,-2.001720,9.988424e-07,-2.001557,1.201458e-05,1227
6,2.485458e+00,72.678154,12.504455,4.304930,18.591523,2.682979,70.192696,5.613415,3.109235e+00,3.560404e+00,...,4.518660e-03,-0.826233,4.517884e-01,-0.798447,3.377328e-01,-1.981263,8.285558e-04,-1.939282,4.776107e-03,1598
7,3.585604e+00,1449.715698,454.827084,144.237851,527.267256,0.693768,1446.130094,3.179516,3.585604e+00,8.624741e+00,...,1.890486e-08,-2.001827,2.290843e-08,-2.001826,-1.605678e-08,-2.001825,1.320402e-08,-2.001822,2.792267e-07,1632
8,2.611106e+00,35.559826,9.352491,5.280550,8.709617,2.021875,32.948720,3.522989,3.632440e+00,3.695240e+00,...,8.646596e-02,-1.235485,9.634463e-02,-1.894259,2.601301e-03,-1.959811,2.241641e-03,-1.997130,2.408746e-04,1920
9,2.697659e+00,232.828339,91.449271,43.159845,93.783926,0.378792,230.130680,2.516485,3.402587e+00,3.630055e+00,...,8.092151e-07,-2.001772,6.463881e-07,-2.001768,-3.574509e-07,-2.001771,4.462634e-07,-2.001481,1.544337e-05,1926


In [8]:
df_meta = pd.concat([ df_meta, agg_train, what, new_feats, agg_autoenc, dtw_train, max_train],axis=1)
min_train.head()

,object_id,no_of_min_0,no_of_min_1,no_of_min_2,no_of_min_3,no_of_min_4,no_of_min_5
0,615,6.0,8.0,6.0,7.0,8.0,7.0
1,713,8.0,7.0,5.0,8.0,8.0,5.0
2,730,11.0,8.0,2.0,1.0,1.0,5.0
3,745,8.0,0.0,1.0,1.0,2.0,3.0
4,1124,8.0,4.0,4.0,2.0,1.0,4.0


In [9]:
#df_meta["hostgal_photoz_certain_mine"] = 2.438*(10**(-20))*np.exp(0.9841*df_meta["distmod"])+0.01821+df_meta["hostgal_photoz"]

df_meta["hostgal_photoz_certain_mine"] = -6.09*(10**(-14))*np.exp(0.6713*df_meta["distmod"])+0.04902+df_meta["hostgal_photoz"]
df_meta["hostgal_photoz_certain_mine"] = df_meta["hostgal_photoz_certain_mine"].replace(0.0490199999999391, 0)
df_meta = df_meta.drop(["object_id","ra", "decl","ddf", "type", "gal_l", "gal_b", "hostgal_specz"], axis=1)
df_meta.drop("hostgal_photoz_certain",axis=1,inplace=True)
df_meta


,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,haversine,hostgal_photoz_certain_mine,latlon1,flux_min,flux_max,...,DTW2-5,DTW3-4,DTW3-5,DTW4-5,no_of_max_0,no_of_max_1,no_of_max_2,no_of_max_3,no_of_max_4,no_of_max_5
0,0.0000,0.0000,0.0000,0.017,92,0.319006,0.000000,-1.528827,-1100.440063,660.626343,...,14751.482711,6153.243755,9220.708458,2937.931631,9.0,14.0,13.0,13.0,14.0,14.0
1,1.6267,0.2552,45.4063,0.007,88,1.698939,0.622654,3.258921,-14.735178,14.770886,...,308.010969,128.780444,250.355226,213.702565,12.0,10.0,11.0,12.0,10.0,9.0
2,0.2262,0.0157,40.2561,0.021,42,1.818030,0.242034,3.128522,-19.159811,47.310059,...,385.492483,139.569941,303.518606,225.333517,21.0,12.0,6.0,6.0,4.0,11.0
3,0.2813,1.1523,40.7951,0.007,90,0.495223,0.282666,6.893743,-15.494463,220.795212,...,409.598389,189.154523,365.628799,369.308356,14.0,3.0,3.0,2.0,4.0,7.0
4,0.2415,0.0176,40.4166,0.024,90,0.395162,0.253559,-1.928064,-16.543753,143.600189,...,400.759720,164.987273,403.816037,344.198020,16.0,6.0,6.0,4.0,2.0,8.0
5,0.0000,0.0000,0.0000,0.020,65,1.862378,0.000000,3.045939,-12.695161,71.678154,...,273.423723,122.991666,241.055044,203.941346,16.0,16.0,15.0,16.0,15.0,13.0
6,0.1820,0.0304,39.7279,0.019,90,0.380970,0.207741,-2.087693,-13.854152,1448.715698,...,1476.659021,639.177658,1001.330093,523.086150,0.0,0.0,0.0,0.0,0.0,0.0
7,0.7014,0.0100,43.1524,0.021,42,0.342552,0.518494,-1.753833,-14.525748,34.559826,...,536.450240,167.947667,424.732214,310.074249,15.0,14.0,16.0,15.0,12.0,14.0
8,0.3229,0.3360,41.1401,0.027,90,1.472458,0.311847,-2.866823,-13.367912,231.828339,...,558.925417,251.025274,475.959889,373.106527,4.0,1.0,1.0,2.0,2.0,3.0
9,0.0000,0.0000,0.0000,0.018,65,1.511114,0.000000,-2.894582,-52.764721,30.658575,...,469.993794,130.921624,400.532933,315.237819,5.0,14.0,8.0,14.0,10.0,11.0


In [10]:
df_meta.columns.values

array(['hostgal_photoz', 'hostgal_photoz_err', 'distmod', 'mwebv',
       'target', 'haversine', 'hostgal_photoz_certain_mine', 'latlon1',
       'flux_min', 'flux_max', 'flux_mean', 'flux_median', 'flux_std',
       'flux_skew', 'flux_err_min', 'flux_err_max', 'flux_err_mean',
       'flux_err_median', 'flux_err_std', 'flux_err_skew',
       'flux_ratio_sq_sum', 'flux_ratio_sq_skew',
       'flux_by_flux_ratio_sq_sum', 'flux_by_flux_ratio_sq_skew',
       'detected_mean', 'flux_diff1', 'flux_diff2', 'flux_diff3',
       'flux_w_mean', 'flux_min_0', 'flux_min_1', 'flux_min_2',
       'flux_min_3', 'flux_min_4', 'flux_min_5', 'flux_max_0',
       'flux_max_1', 'flux_max_2', 'flux_max_3', 'flux_max_4',
       'flux_max_5', 'flux_mean_0', 'flux_mean_1', 'flux_mean_2',
       'flux_mean_3', 'flux_mean_4', 'flux_mean_5', 'flux_median_0',
       'flux_median_1', 'flux_median_2', 'flux_median_3', 'flux_median_4',
       'flux_median_5', 'flux_std_0', 'flux_std_1', 'flux_std_2',
       'flux_s

## Deep learning
### With keras and tensorflow 
#### Got most of the code from Siddhartha's public kernel, https://www.kaggle.com/meaninglesslives/simple-neural-net-for-time-series-classification?fbclid=IwAR1UJHWeMYPjHVw4E-3HxR7gip-cSQNcyq1q1VadOATDee-Se3MpnUD8o2E

In [11]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.preprocessing import StandardScaler
from functools import partial, wraps

from sklearn.model_selection import StratifiedKFold
xgb_params = {    
        'objective': 'multi:softprob', 
        'eval_metric': 'mlogloss', 
        'silent': True, 
        'num_class':14,
        
        'booster': 'gbtree',
        'n_jobs': 8,
        'n_estimators': 500,
        'tree_method': 'hist',
        'grow_policy': 'lossguide',
        'base_score': 0.25,
        'max_depth': 7,
        'max_delta_step': 2,  #default=0
        'learning_rate': 0.03,
        'max_leaves': 11,
        'min_child_weight': 64,
        'gamma': 0.1, # default=
        'subsample': 0.7,
        'colsample_bytree': 0.68,
        'reg_alpha': 0.01, # default=0
        'reg_lambda': 10., # default=1
        'seed': 538
}

lgbm_params = {
            'device': 'cpu', 
            'objective': 'multiclass', 
            'num_class': 14, 
            'boosting_type': 'gbdt', 
            'n_jobs': -1, 
            'max_depth': 7, 
            'n_estimators': 1000, 
            'subsample_freq': 2, 
            'subsample_for_bin': 10000, 
            'min_data_per_group': 100, 
            'max_cat_to_onehot': 4, 
            'cat_l2': 1.0, 
            'cat_smooth': 59.5, 
            'max_cat_threshold': 32, 
            'metric_freq': 10, 
            'verbosity': -1, 
            'metric': 'multi_logloss', 
            'xgboost_dart_mode': True, 
            'uniform_drop': False, 
            'colsample_bytree': 0.5, 
            'drop_rate': 0.173, 
            'learning_rate': 0.0367, 
            'max_drop': 5, 
            'min_child_samples': 10, 
            'min_child_weight': 100.0, 
            'min_split_gain': 0.1, 
            'num_leaves': 7, 
            'reg_alpha': 0.1, 
            'reg_lambda': 0.00023, 
            'skip_drop': 0.44, 
            'subsample': 0.75
}

In [12]:
def multi_weighted_logloss(y_true, y_preds, classes, class_weights):
    """
    refactor from
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weights[k] for k in sorted(class_weights.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgbm_multi_weighted_logloss(y_true, y_preds):
    """
    refactor from
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """  
    # Taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weights = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}

    loss = multi_weighted_logloss(y_true, y_preds, classes, class_weights)
    return 'wloss', loss, False


def save_importances(importances_):
    mean_gain = importances_[['gain', 'feature']].groupby('feature').mean()
    importances_['mean_gain'] = importances_['feature'].map(mean_gain['gain'])
    return importances_

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(kind="svm", random_state=12)

def lgbm_modeling_cross_validation(params,
                                   full_train, 
                                   y, 
                                   classes, 
                                   class_weights, 
                                   nr_fold=20, 
                                   random_state=1):
    
    

    # Compute weights
    w = y.value_counts()
    weights = {i : np.sum(w) / w[i] for i in w.index}

    clfs = []
    importances = pd.DataFrame()
    folds = StratifiedKFold(n_splits=nr_fold, 
                            shuffle=True, 
                            random_state=random_state)
    
    oof_preds = np.zeros((len(full_train), np.unique(y).shape[0]))
    
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        
        trn_x, trn_y = full_train.iloc[trn_], y.iloc[trn_]
        val_x, val_y = full_train.iloc[val_], y.iloc[val_]
        
        trn_x_resampled, trn_y_resampled = sm.fit_sample(trn_x,trn_y)
        trn_x_resampled = pd.DataFrame(trn_x_resampled)
        trn_x_resampled.columns = trn_x.columns
        #trn_x_resampled = trn_x
        #trn_y_resampled = trn_y
        trn_y_resampled = pd.Series(trn_y_resampled)
        clf = LGBMClassifier(**params)
        clf.fit(
            trn_x_resampled, trn_y_resampled,
            eval_set=[(trn_x_resampled, trn_y_resampled), (val_x, val_y)],
            eval_metric=lgbm_multi_weighted_logloss,
            verbose=100,
            early_stopping_rounds=50,
            sample_weight=trn_y_resampled.map(weights)
        )
        clfs.append(clf)

        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        print('no {}-fold loss: {}'.format(fold_ + 1, 
              multi_weighted_logloss(val_y, oof_preds[val_, :], 
                                     classes, class_weights)))
    
        imp_df = pd.DataFrame({
                'feature': full_train.columns,
                'gain': clf.feature_importances_,
                'fold': [fold_ + 1] * len(full_train.columns),
                })
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

    score = multi_weighted_logloss(y_true=y, y_preds=oof_preds, 
                                   classes=classes, class_weights=class_weights)
    print('MULTI WEIGHTED LOG LOSS: {:.5f}'.format(score))
    df_importances = save_importances(importances_=importances)
    df_importances.to_csv('lgbm_importances.csv', index=False)
    
    return clfs, score


In [ ]:
# from functools import partial, wraps

class_names = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
class_weights = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}

scoring_methods =('accuracy')
train_mean = df_meta.mean(axis=0)
df_meta.fillna(0, inplace=True)

# modeling from CV
# Feature Scaling
 
X = (df_meta[df_meta.columns.difference(['target'])])
#X = clean_dataset(X)
y = (df_meta["target"])
clfs, score = lgbm_modeling_cross_validation(lgbm_params, X, y, classes=class_names, class_weights=class_weights, nr_fold=10, random_state=1)



In [12]:
def haversine_plus(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees) from 
    #https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
    """
    #Convert decimal degrees to Radians:
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    dlat = np.subtract(lat2, lat1)

    a = np.add(np.power(np.sin(np.divide(dlat, 2)), 2),  
                          np.multiply(np.cos(lat1), 
                                      np.multiply(np.cos(lat2), 
                                                  np.power(np.sin(np.divide(dlon, 2)), 2))))
    
    haversine = np.multiply(2, np.arcsin(np.sqrt(a)))
    return {
        'haversine': haversine, 
        'latlon1': np.subtract(np.multiply(lon1, lat1), np.multiply(lon2, lat2)), 
   }

def process_meta(filename):
    meta_df = pd.read_csv(filename)
    meta_df["distmod"].fillna(0, inplace=True)
    meta_dict = dict()
    # distance
    meta_dict.update(haversine_plus(meta_df['ra'].values, meta_df['decl'].values, 
                   meta_df['gal_l'].values, meta_df['gal_b'].values))
    #
    meta_dict['hostgal_photoz_certain'] = np.multiply(
            meta_df['hostgal_photoz'].values, 
             np.exp(meta_df['hostgal_photoz_err'].values))
    #meta_df["hostgal_photoz"].fillna(0, inplace=True)
    #meta_df["hostgal_photoz_err"].fillna(0, inplace=True)
    meta_df["hostgal_photoz_certain_mine"] = -6.09*(10**(-14))*np.exp(0.6713*meta_df["distmod"])+0.04902+meta_df["hostgal_photoz"]
    meta_df["hostgal_photoz_certain_mine"] = meta_df["hostgal_photoz_certain_mine"].replace(0.0490199999999391, 0)
    meta_df = pd.concat([meta_df, pd.DataFrame(meta_dict, index=meta_df.index)], axis=1)
    return meta_df

df_meta_test = process_meta("all/test_set_metadata.csv")
df_meta_test

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,hostgal_photoz_certain_mine,haversine,hostgal_photoz_certain,latlon1
0,13,34.453125,-5.229529,169.987075,-59.956185,1,0.3048,0.3193,0.0542,41.1123,0.019,0.309358,1.851382,0.337084,3.049709
1,14,33.398438,-4.331149,167.226341,-59.936551,1,NaN,0.6323,0.0179,42.8774,0.018,0.488490,1.855173,0.643720,3.009107
2,17,348.529419,-61.755440,321.293980,-51.763351,1,NaN,0.8297,0.0605,43.6000,0.016,0.565504,0.309914,0.881446,-1.490290
3,23,34.804688,-5.829153,171.307861,-60.174401,1,NaN,0.6533,0.1479,42.9640,0.023,0.497947,1.845038,0.757434,3.078301
4,34,351.321442,-64.198746,317.458993,-50.429931,1,0.4557,0.4617,0.0122,42.0540,0.023,0.399772,0.391772,0.467367,-1.993709
5,35,35.332031,-5.979157,172.286722,-59.931743,1,NaN,0.8388,0.0375,43.6290,0.022,0.568447,1.848367,0.870852,3.080958
6,43,0.574468,-45.981140,327.041068,-68.778764,1,NaN,0.6669,0.0546,43.0186,0.006,0.503917,0.494902,0.704325,6.843860
7,50,0.574468,-45.981140,327.041068,-68.778764,1,NaN,1.4663,0.0529,45.1281,0.006,0.641648,0.494902,1.545956,6.843860
8,60,346.562500,-63.448284,320.824720,-49.866957,1,NaN,0.9462,0.0116,43.9519,0.021,0.598543,0.337900,0.957240,-1.824732
9,69,349.160583,-64.760857,318.219706,-49.458924,1,NaN,1.0432,0.1092,44.2138,0.020,0.619295,0.389474,1.163570,-2.093676


In [ ]:
df_meta_test.to_csv("test/neg_flux/df_meta.csv")

In [13]:
class_names = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
head2 = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95, "object_id", 99]
#df_meta_test["hostgal_specz"].fillna((df_meta_test["hostgal_photoz"]), inplace=True)
df_meta_test = pd.read_csv("test/neg_flux/df_meta.csv")
df_meta_test = df_meta_test.drop(["ra", "ddf", "decl", "gal_l", "hostgal_specz", "gal_b"], axis=1)
df_meta_test["distmod"].fillna(0, inplace=True)
objs = df_meta_test["object_id"]
df_meta_test.set_index("object_id", inplace=True)

agg_test = pd.read_csv("test/neg_flux/agg_test_corr.csv")
unorderd_objs = agg_test["object_id"]
agg_test.set_index("object_id", inplace=True)

max_test = pd.read_csv("test/neg_flux/max_test.csv")
max_test["object_id"] = unorderd_objs
max_test.set_index("object_id", inplace=True)

mjd_diff_test = pd.read_csv("test/neg_flux/mjd_diffs.csv")
mjd_diff_test["object_id"] = unorderd_objs
mjd_diff_test.set_index("object_id", inplace=True)

dtw_test = pd.read_csv("test/neg_flux/dtw_test_full.csv")
dtw_test["object_id"] = unorderd_objs
dtw_test.set_index("object_id", inplace=True)



In [14]:
what_test = pd.read_csv("test/neg_flux/encoder_test.csv")
what_test = what_test[:3492890]
what_test.drop("Unnamed: 0",axis=1,inplace=True)
what_test.columns = ["Autoencoder" + str(s) for s in range(0,16)]
what_test["object_id"] = objs
what_test.set_index("object_id", inplace=True)
what_test

,Autoencoder0,Autoencoder1,Autoencoder2,Autoencoder3,Autoencoder4,Autoencoder5,Autoencoder6,Autoencoder7,Autoencoder8,Autoencoder9,Autoencoder10,Autoencoder11,Autoencoder12,Autoencoder13,Autoencoder14,Autoencoder15
object_id,,,,,,,,,,,,,,,,
13,0.0,37.185764,182.10419,157.359880,103.555250,183.793550,0.0,254.87636,125.211570,0.0,112.452010,0.000000,0.0,0.0,66.724160,207.84607
14,0.0,35.927464,175.15640,151.204420,99.814310,176.513370,0.0,244.75363,120.184070,0.0,108.676100,0.000000,0.0,0.0,63.727220,200.18317
17,0.0,38.322712,186.85983,161.314180,106.726130,188.309900,0.0,261.00018,128.104280,0.0,116.206406,0.000000,0.0,0.0,67.916230,213.54443
23,0.0,36.221382,176.36714,151.586400,100.433710,177.070700,0.0,246.21740,120.614716,0.0,109.107635,0.000000,0.0,0.0,64.115560,200.86597
34,0.0,40.471317,202.19327,178.534100,115.122330,206.884250,0.0,286.73856,141.262860,0.0,125.801550,0.000000,0.0,0.0,76.285120,232.06676
35,0.0,36.537384,178.88994,153.879970,101.641785,179.879530,0.0,250.14923,122.902370,0.0,110.198830,0.000000,0.0,0.0,65.617880,203.65173
43,0.0,39.348040,194.61180,167.573470,111.874050,196.265900,0.0,272.72403,133.720690,0.0,120.289215,0.000000,0.0,0.0,71.773230,222.30925
50,0.0,39.085205,193.11026,166.369340,111.042890,194.738250,0.0,270.76077,132.636370,0.0,119.584340,0.000000,0.0,0.0,71.220560,220.72313
60,0.0,38.867330,190.19525,163.830980,108.493835,191.482800,0.0,265.66900,130.497910,0.0,117.722600,0.000000,0.0,0.0,69.378090,216.86255


In [15]:
new_feats = pd.read_csv("test/neg_flux/new_feats.csv")
new_feats.set_index("object_id",inplace=True)

,object_id,new_feats0,new_feats1,new_feats2,new_feats3,new_feats4,new_feats5,new_feats6,new_feats7,new_feats8,...,new_feats62,new_feats63,new_feats64,new_feats65,new_feats66,new_feats67,new_feats68,new_feats69,new_feats70,new_feats71
0,13,7.551563,-0.225294,-0.158192,-12.249563,-0.700870,2.679826,NaN,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,14,14.465278,2.266812,2.050447,-5.444651,0.811231,3.600111,NaN,0.033058,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,17,7.867962,-0.513157,-0.111748,-14.202744,-1.070882,3.090153,NaN,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,23,11.378603,-0.032929,0.110454,-12.631923,0.094735,2.954068,NaN,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,34,8.882544,-0.061707,-0.072111,-11.584136,-0.702452,3.143884,NaN,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,35,8.650745,0.130447,0.017391,-5.975548,0.712239,2.231072,NaN,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,43,14.581906,1.521868,0.785032,-6.696694,1.034880,3.406364,NaN,0.040323,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,50,12.179604,0.689169,0.128519,-11.342772,0.352836,3.601927,NaN,0.016129,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,60,15.975101,0.117300,-0.245359,-9.571251,1.791609,3.308378,NaN,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,69,11.440970,-0.159306,-0.095366,-13.369169,-0.184607,3.224268,NaN,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
full_test = pd.concat([df_meta_test,new_feats, what_test, dtw_test, max_test, agg_test],axis=1)
full_test.fillna(0, inplace=True)
full_test.reset_index(inplace=True)
full_test.drop(["object_id", "Unnamed: 0"], axis=1, inplace=True)
full_test.sort_index(axis=1,inplace=True)
full_test

,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",...,new_feats70,new_feats71,new_feats8,new_feats9,no_of_max_0,no_of_max_1,no_of_max_2,no_of_max_3,no_of_max_4,no_of_max_5
0,29.002872,37.684424,-0.247161,0.147622,92.039691,79.990818,10.567411,3.117684,164.640622,139.733762,...,0.000000,0.000000,0.0,0.0,18.0,7.0,5.0,4.0,5.0,13.0
1,22.708482,26.159786,0.099267,0.502325,13.685195,27.630359,13.429230,3.109318,31.012898,33.427073,...,0.000000,0.000000,0.0,0.0,21.0,10.0,9.0,12.0,12.0,13.0
2,6.030863,46.450439,0.590786,-0.427429,53.366119,41.192677,4.127763,1.710048,58.607170,42.353288,...,0.000000,0.000000,0.0,0.0,19.0,12.0,11.0,16.0,15.0,16.0
3,0.776942,14.926490,-0.580529,-0.058688,2.371129,22.476077,1.429683,-0.000571,34.447730,14.240274,...,0.000000,0.000000,0.0,0.0,19.0,12.0,15.0,12.0,12.0,14.0
4,65.064185,55.191226,7.457426,2.135457,202.641917,191.410916,16.393709,4.133929,425.940450,414.455380,...,0.000000,0.000000,0.0,0.0,16.0,1.0,1.0,1.0,2.0,9.0
5,12.282884,22.620372,1.225421,-0.094534,5.438267,7.448297,1.867587,-0.883637,66.203272,55.011886,...,0.000000,0.000000,0.0,0.0,23.0,13.0,12.0,5.0,7.0,11.0
6,12.923340,18.602257,0.781549,0.027175,22.635297,21.625244,2.001422,0.423699,25.728478,19.711853,...,0.000000,0.000000,0.0,0.0,23.0,13.0,11.0,11.0,17.0,15.0
7,19.609702,2.330404,0.752670,-0.014739,1.211557,15.708252,1.843672,0.000526,22.003983,23.813090,...,0.000000,0.000000,0.0,0.0,18.0,17.0,13.0,16.0,14.0,17.0
8,14.553430,14.701939,-0.258688,0.163146,8.271726,6.506563,2.859515,-0.320960,56.608444,47.318795,...,0.000000,0.000000,0.0,0.0,19.0,15.0,13.0,7.0,6.0,12.0
9,33.172974,20.737749,-0.253322,0.258366,28.220237,42.894740,1.900564,0.917209,49.124642,50.017043,...,0.000000,0.000000,0.0,0.0,16.0,18.0,11.0,4.0,9.0,14.0


In [18]:
full_test.drop("hostgal_photoz_certain",axis=1,inplace=True)

In [22]:
# Make predictions

preds = None
for clf in clfs:
    if(preds is None):
        preds = clf.predict_proba(full_test)
    else:
        preds+= clf.predict_proba(full_test)
preds = preds/ len(clfs)

#preds_99 =np.where(np.max(preds)>0.9 , 0.01, 0.1)
# Store predictions
preds_df = pd.DataFrame(preds, columns=class_names)
preds_df['object_id'] = objs

In [23]:
preds_df = preds_df[['object_id',6,15,16,42,52,53, 62,64,65,67,88, 90,92,95]]
preds_df.columns = ['object_id',
'class_6',
'class_15',
'class_16',
'class_42',
'class_52',
'class_53',
'class_62',
'class_64',
'class_65',
'class_67',
'class_88',
'class_90',
'class_92',
'class_95']
preds_df.head()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95
0,13,0.000253,0.001377,0.000085,0.394585,0.548358,0.000581,0.042881,0.000237,0.000106,0.002263,0.000383,0.008135,0.000225,0.000532
1,14,0.000675,0.014659,0.000416,0.447171,0.064436,0.001553,0.076002,0.001301,0.000734,0.191134,0.004374,0.175076,0.000733,0.021736
2,17,0.000967,0.012840,0.000651,0.396002,0.125789,0.002171,0.067403,0.007055,0.000735,0.133971,0.008551,0.181296,0.000856,0.061714
3,23,0.000656,0.004145,0.000418,0.217880,0.064404,0.001490,0.080036,0.005884,0.000497,0.401244,0.001141,0.136577,0.000523,0.085104
4,34,0.000294,0.003042,0.000126,0.119391,0.535880,0.000654,0.035202,0.000300,0.000210,0.043758,0.000241,0.259917,0.000224,0.000763


In [24]:
def GenUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

y = pd.DataFrame()
y['mymean'] = preds_df[feats].mean(axis=1)
y['mymedian'] = preds_df[feats].median(axis=1)
y['mymax'] = preds_df[feats].max(axis=1)

class_99 = GenUnknown(y)
preds_df["class_99"] = class_99

In [25]:
preds_df.set_index("object_id", inplace=True)

In [26]:
preds_df.to_csv("skase.csv")